<H1><b>Segmenting and Clustering Neighborhoods in Toronto</b></H1>

In [47]:
#import libraries
%pip install beautifulsoup4
from bs4 import BeautifulSoup
%pip install lxml
import requests
import pandas as pd
import numpy as np


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


<H2>Scrapping the Wikipedia Table as DataFrame</H2>

In [48]:
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'lxml')
df=pd.read_html(source)
df=df[0]
df=df.rename(columns={'Postal Code':'PostalCode'}) #Changing Column Names
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<H3>...Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.</H3>

In [49]:
df=df[df.Borough != 'Not assigned']
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<H3>...These two rows will be combined into   one row with the neighborhoods separated with a comma as shown in row 11 in the above table.</H3>

In [50]:
df=df.groupby(['PostalCode','Borough'], sort=False).agg(','.join)
df.reset_index(inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<H3>...If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.</H3>

In [51]:
df['Neighbourhood']=np.where(df['Neighbourhood']=='Not assigned',df['Borough'],df['Neighbourhood'])
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<H3>...use the .shape method to print the number of rows of your dataframe</H3>

In [52]:
df.shape

(103, 3)

<H3>Use the Geocoder package or the csv file to create the following dataframe</H3>

In [53]:
codf=pd.read_csv('https://cocl.us/Geospatial_data') #reading csv file from url
codf=codf.rename(columns={'Postal Code':'PostalCode'}) #Changing Column Names
codf #showing csv file as dataframe

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [56]:
df = pd.merge(df, codf,on ='PostalCode')   #adding Latitude and Longitude data to main dataframe 
df             

,PostalCode,Borough,Neighbourhood,Latitude_x,Longitude_x,Latitude_y,Longitude_y
0,M3A,North York,Parkwoods,43.753259,-79.329656,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,43.662301,-79.389494
...,...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,43.636258,-79.498509
